# Overview

Interfereometer data (visibilities) and single dish data (spectrum).

MSv2 and CASA documentation used in creating the schema:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data
- VLBI MSv2 extension: https://casacore.github.io/casacore-notes/265.pdf

We reused descriptions from the MSv2 (add thank you).


Version numbering explained.



<!--Link to google drawing: https://docs.google.com/drawings/d/1afPe5oro26NMTkAKpK9iif0adNA0B4R9otLookOixvI/edit?usp=sharing -->

Will eventually replace Python-casacore.
Asdm library does not exist yet.

https://docs.google.com/drawings/d/1P9AI3D4VzPGw8O72dPBkz1iED8-ITM96anLeru0dvgM/edit?usp=sharing

<img src="https://docs.google.com/drawings/d/e/2PACX-1vQVgjF5xNeIv8gpi2G3R8JXw2bNkVIUXdizIZluCGdnHc4z79ryW2fNUycJAd_CQh9sXLwdlx1oiAAX/pub?w=690&amp;h=510">

## Processing Set Data Structure

## Measurement Set v4

### Measurement Set v2 Deprecated 

### Data Groups (optional)

Data reuse.